In [105]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
from datetime import datetime

## Задание 1 (с доп частью)

In [103]:
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/all/'

In [104]:
def all_links(URL):
    links = pd.DataFrame()

    res = requests.get(URL)
    soup = BeautifulSoup(res.text, 'html.parser')
    posts = soup.find_all('article', class_='post')

    for post in posts:

        post_id = post.parent.attrs.get('id')
        if not post_id:
            continue

        article_link = post.find('a', class_='btn').get('href')
        article_res = requests.get(article_link)
        soup_article = BeautifulSoup(article_res.text, 'html.parser')
        articles = soup_article.find_all('div', class_='post__text')
        
        for article in articles:

            if any([desired in article.text.lower() for desired in KEYWORDS]):
                
                title = post.find('a', class_='post__title_link')
                link = title.get('href')
                date = post.find('span', class_='post__time').text
                text = article.text
                
                row = {'date': date, 'title': title.text, 'link': link, 'text': text}

                links = pd.concat([links, pd.DataFrame([row])])
        
    return links

all_links('https://habr.com/ru/all/')

,date,title,link,text
0,сегодня в 10:08,Мы создали Web приложение для определения лиц ...,https://habr.com/ru/post/547038/,ВведениеОсновная цель - обнаружение лица и мас...
0,сегодня в 09:12,Мониторинг производительности приложений и мет...,https://habr.com/ru/post/546772/,"Привет, Habr! Я уже рассказывал про AIOps и ме..."
0,сегодня в 03:42,ossh: параллельное выполнение команд на многих...,https://habr.com/ru/post/546998/,Иногда бывает нужно запустить патч Бармина как...


## Задание 2

In [100]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
EMAIL = ['xxx@x.ru', 'yyy@y.com']

headers = {
    "Vaar-Version": "0",
    "Vaar-Header-App-Product-Name": "hackcheck-web-avast",
    "Vaar-Header-App-Build-Version": "1.0.0"
}

In [101]:
def get_email_information(EMAIL, URL):

    payload = {
        "emailAddresses": EMAIL
        }

    payload = json.dumps(payload)

    res = requests.post(URL, payload, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    information = json.loads(soup.text)

    df = pd.DataFrame()

    for site in information['data']:
        for email in information['data'][site]:
            breach_id = str(information['data'][site][email][0]['breachId'])
            site = information['breaches'][breach_id]['site']
            date = information['breaches'][breach_id]['publishDate']
            description = information['breaches'][breach_id]['description']
            row = {'email': email, 'publish_date': date, 'source': site, 'description': description}
            df = pd.concat([df, pd.DataFrame([row])]).reset_index(drop=True)

    return df

In [102]:
get_email_information(EMAIL, URL)

,email,publish_date,source,description
0,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
1,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
2,yyy@y.com,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
3,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
4,yyy@y.com,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
5,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
6,yyy@y.com,2017-12-01T00:00:00Z,creocommunity.com,"At an unconfirmed date, Creo Community's user ..."
7,yyy@y.com,2019-07-11T00:00:00Z,medicaresupplement.com,"In May 2019, a security researcher discovered ..."
8,yyy@y.com,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
9,yyy@y.com,2020-11-19T00:00:00Z,123rf.com,"In March 2020, the stock image agency 123RF wa..."


## Задание 2 (доп часть)

In [18]:
import vk_api

In [164]:
GROUP = 'netology'
count = 50 #количество записей

In [165]:
vk_session = vk_api.VkApi('+79046048414', <password>)
vk_session.auth()

vk = vk_session.get_api()

In [166]:
def get_group_post(group, count):

    search_id = vk.utils.resolveScreenName(screen_name=GROUP)
    id = search_id['object_id']*(-1)
    posts = vk.wall.get(owner_id=id, count=count)
    df_posts = pd.DataFrame()

    for i in range(count):

        date = posts['items'][i]['date']
        date = datetime.fromtimestamp(date)
        date = str(date.year) + '-' + str(date.month) + '-' + str(date.day) + ' ' + str(date.hour) + ':' + str(date.minute)
        text = posts['items'][i]['text']
        row = {'date': date, 'text': text}

        df_posts = pd.concat([df_posts, pd.DataFrame([row])]).reset_index(drop=True)

    df_posts.index = df_posts.index + 1

    return df_posts

In [167]:
get_group_post(GROUP, count)

,date,text
1,2021-3-10 17:40,"Проверим, насколько хорошо вы знаете нас 😉 \nЗ..."
2,2021-3-14 12:49,"Говорят, лидерами не рождаются, а становятся 😎..."
3,2021-3-13 12:53,"Выстраивать отношения с коллегами, решать зада..."
4,2021-3-12 17:47,"В эфире #пятничнаябеседка. Здесь мы обсуждаем,..."
5,2021-3-12 11:11,"Настраиваем рекламные кабинеты, пишем сценарий..."
6,2021-3-11 17:53,Даже самый уверенный в себе человек испытывает...
7,2021-3-11 11:40,Интернет — пространство безграничных возможнос...
8,2021-3-10 10:53,🔥 15 марта стартует бесплатный курс «Как стать...
9,2021-3-9 17:53,"Поиск изображений для своего блога, сайта или ..."
10,2021-3-8 17:13,В марте на рынке вакансий настоящий бум — вспл...
